# Project: Sentiment Analysis on Predoct Reviews

[Dataset name (Women’s Clothing E-Commerce Reviews)](https://www.kaggle.com/datasets/nicapotato/womens-ecommerce-clothing-reviews)

## Imports

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import (
    DistilBertTokenizerFast,
    TFDistilBertForSequenceClassification
)
import matplotlib.pyplot as plt
import kagglehub
from transformers import RobertaTokenizerFast
from transformers import TFRobertaForSequenceClassification

# Config/Hyper Params

In [17]:
# Model
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS = 3

# Training
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 2e-5

# Tokenization
MAX_LEN = 128

# Random seed
SEED = 42

## Load the Dataset

In [18]:
path = kagglehub.dataset_download("nicapotato/womens-ecommerce-clothing-reviews")

df = pd.read_csv(path + '/Womens Clothing E-Commerce Reviews.csv')

df.head()


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


## Preprocessing

In [20]:
df = df.drop(columns=["Unnamed: 0", "Clothing ID", "Age", "Positive Feedback Count", \
"Division Name", "Department Name", "Class Name"]) #removing all unecessary data
#df = df.drop(columns=["Recommended IND"]) #If conclusively not needed

In [21]:
#df = df.dropna(subset=["Review Text"])
df = df.dropna(subset=["Title", "Review Text"], how='all') #To filter out completely empty reviews
df = df[(df["Review Text"].str.strip() != "") & (df["Title"].str.strip() != "")] #In case both are whitespace

In [22]:
#df["text"] = df["Title"].fillna("") + ". " + df["Review Text"]
df["text"] = df["Title"].fillna("").str.strip() + ". " + df["Review Text"].fillna("").str.strip() #In case we decide to only throw out empty reviews

In [23]:
def rating_to_sentiment(r):
    if r <= 2:
        return 0  # negative
    elif r == 3:
        return 1  # neutral
    else:
        return 2  # positive

df["sentiment"] = df["Rating"].apply(rating_to_sentiment)
df["sentiment"].value_counts()


sentiment
2    17449
1     2823
0     2370
Name: count, dtype: int64

In [24]:
#print(df[df["text"].str.split().str.len() < 5]) #Checking what we're missing
df = df[df["text"].str.split().str.len() >= 5] #Remove short reviews

In [25]:
df[["text", "sentiment"]].head()

,text,sentiment
0,. Absolutely wonderful - silky and sexy and co...,2
1,. Love this dress! it's sooo pretty. i happe...,2
2,Some major design flaws. I had such high hopes...,1
3,"My favorite buy!. I love, love, love this jump...",2
4,Flattering shirt. This shirt is very flatterin...,2


In [26]:
df["sentiment"].value_counts(normalize=True)

sentiment
2    0.770580
1    0.124696
0    0.104724
Name: proportion, dtype: float64

In [27]:
df[["text", "sentiment"]].head()
df["sentiment"].value_counts(normalize=True)


sentiment
2    0.770580
1    0.124696
0    0.104724
Name: proportion, dtype: float64

## Train / Validation / Test Split

In [28]:
X = df["text"].values
y = df["sentiment"].values

In [29]:
import numpy as np

np.random.seed(SEED)

indices = np.arange(len(X))
np.random.shuffle(indices)

train_end = int(0.8 * len(X))
val_end = int(0.9 * len(X))

train_idx = indices[:train_end]
val_idx = indices[train_end:val_end]
test_idx = indices[val_end:]

X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val = X[val_idx], y[val_idx]
X_test, y_test = X[test_idx], y[test_idx]

In [30]:
def show_distribution(name, labels):
    unique, counts = np.unique(labels, return_counts=True)
    print(name, dict(zip(unique, counts)))

show_distribution("Train", y_train)
show_distribution("Validation", y_val)
show_distribution("Test", y_test)


Train {np.int64(0): np.int64(1916), np.int64(1): np.int64(2227), np.int64(2): np.int64(13961)}
Validation {np.int64(0): np.int64(234), np.int64(1): np.int64(301), np.int64(2): np.int64(1728)}
Test {np.int64(0): np.int64(220), np.int64(1): np.int64(294), np.int64(2): np.int64(1750)}


## tokenizer + model skeleton

In [31]:
tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_NAME)


### Tokenizing

In [32]:
def tokenize(text):
    return tokenizer(
        list(text),
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="tf"
    )

train_encodings = tokenize(X_train)
val_encodings = tokenize(X_val)
test_encodings = tokenize(X_test)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


### TensorFlow Datasets

In [33]:
train_ds = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(1000).batch(BATCH_SIZE)

val_ds = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(BATCH_SIZE)


### Load Model

In [34]:
model = TFRobertaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Compile the Model (Fine-Tuning Setup)

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


### Fine-Tune the Model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/3


### Evaluate on the Test Set - One time

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

283/283 [==============================] - 25s 90ms/step - loss: 0.7011 - accuracy: 0.7703
Test accuracy: 0.7703180313110352


### Save the Model

In [ ]:
model.save_pretrained("distilbert_sentiment_model")
tokenizer.save_pretrained("distilbert_sentiment_model")

### Prediction



In [ ]:
label_map = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

def predict_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensors="tf",
        truncation=True,
        padding=True,
        max_length=MAX_LEN
    )

    logits = model(**inputs).logits
    probs = tf.nn.softmax(logits, axis=1).numpy()[0]
    pred = np.argmax(probs)

    return {
        "label": label_map[pred],
        "probabilities": {
            "negative": float(probs[0]),
            "neutral": float(probs[1]),
            "positive": float(probs[2])
        }
    }


### Testing It

In [ ]:
predict_sentiment("This product was okay, but nothing special.")


{'label': 'positive',
 'probabilities': {'negative': 0.07995282858610153,
  'neutral': 0.13062600791454315,
  'positive': 0.7894211411476135}}

In [ ]:
predict_sentiment("Absolutely terrible quality. Waste of money.")

{'label': 'positive',
 'probabilities': {'negative': 0.07995282858610153,
  'neutral': 0.13062599301338196,
  'positive': 0.7894211411476135}}

In [ ]:
predict_sentiment("I love this dress, it fits perfectly.")

{'label': 'positive',
 'probabilities': {'negative': 0.07995282858610153,
  'neutral': 0.13062600791454315,
  'positive': 0.7894211411476135}}